# 데이터 준비

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import lightgbm as lgb
!pip install catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
total = pd.read_csv('/content/drive/MyDrive/21-2 kubig 분류예측스터디/데이콘 가스공급량/사용 데이터/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [ ]:
total.head(10)

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105
5,2013-01-01,6,A,2390.961
6,2013-01-01,7,A,2378.457
7,2013-01-01,8,A,2518.921
8,2013-01-01,9,A,2706.481
9,2013-01-01,10,A,2832.057


In [ ]:
total.isnull().sum()

연월일    0
시간     0
구분     0
공급량    0
dtype: int64

In [ ]:
total['공급량'].describe()

count    368088.000000
mean        948.100037
std         927.211578
min           1.378000
25%         221.973000
50%         637.014000
75%        1398.919000
max       11593.617000
Name: 공급량, dtype: float64

# 변수 생성(파생 변수 + 외부데이터 등)

## 기온 데이터 불러오기

In [ ]:
PATH = '/content/drive/MyDrive/21-2 kubig 분류예측스터디/데이콘 가스공급량/사용 데이터/'

In [ ]:
weather_2013 = pd.read_csv(PATH + '기상정보2013.csv', encoding='cp949')
weather_2014 = pd.read_csv(PATH + '기상정보2014.csv', encoding='cp949')
weather_2015 = pd.read_csv(PATH + '기상정보2015.csv', encoding='cp949')
weather_2016 = pd.read_csv(PATH + '기상정보2016.csv', encoding='cp949')
weather_2017 = pd.read_csv(PATH + '기상정보2017.csv', encoding='cp949')
weather_2018 = pd.read_csv(PATH + '기상정보2018.csv', encoding='cp949')
weather_2019 = pd.read_csv(PATH + '기상정보2019.csv', encoding='cp949')

In [ ]:
def df_duplicate(df):
  new_df = pd.DataFrame()
  for _ in range(7):
    new_df = pd.concat([new_df, df])
  return new_df

In [ ]:
weather_2013_total = df_duplicate(weather_2013)
weather_2014_total = df_duplicate(weather_2014)
weather_2015_total = df_duplicate(weather_2015)
weather_2016_total = df_duplicate(weather_2016)
weather_2017_total = df_duplicate(weather_2017)
weather_2018_total = df_duplicate(weather_2018)
weather_2019_total = df_duplicate(weather_2019)

In [ ]:
weather_total = pd.concat([weather_2013_total, weather_2014_total, weather_2015_total, weather_2016_total, weather_2017_total, weather_2018_total])

In [ ]:
weather_total

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),적설(cm),전운량(10분위)
0,108,서울,2013-01-01 00:00,-8.3,NaN,3.6,56.0,NaN,6.4,3.0
1,108,서울,2013-01-01 01:00,-8.5,NaN,4.0,57.0,NaN,6.4,NaN
2,108,서울,2013-01-01 02:00,-8.4,NaN,4.0,60.0,NaN,6.4,NaN
3,108,서울,2013-01-01 03:00,-8.1,NaN,3.4,58.0,NaN,6.4,6.0
4,108,서울,2013-01-01 04:00,-8.2,NaN,3.8,58.0,NaN,6.4,6.0
...,...,...,...,...,...,...,...,...,...,...
8755,108,서울,2018-12-31 19:00,-3.1,NaN,2.1,43.0,NaN,NaN,0.0
8756,108,서울,2018-12-31 20:00,-3.7,NaN,0.8,39.0,NaN,NaN,5.0
8757,108,서울,2018-12-31 21:00,-4.6,NaN,1.1,44.0,NaN,NaN,0.0
8758,108,서울,2018-12-31 22:00,-5.4,NaN,1.3,46.0,NaN,NaN,0.0


In [ ]:
total['연월일'] = pd.to_datetime(total['연월일'])
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [ ]:
total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,A,2497.129,2013,1,1,1
1,2013-01-01,2,A,2363.265,2013,1,1,1
2,2013-01-01,3,A,2258.505,2013,1,1,1
3,2013-01-01,4,A,2243.969,2013,1,1,1
4,2013-01-01,5,A,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,H,681.033,2018,12,31,0
368084,2018-12-31,21,H,669.961,2018,12,31,0
368085,2018-12-31,22,H,657.941,2018,12,31,0
368086,2018-12-31,23,H,610.953,2018,12,31,0


In [ ]:
total['temp'] = weather_total['기온(°C)'].values

In [ ]:
total

,연월일,시간,구분,공급량,year,month,day,weekday,temp
0,2013-01-01,1,A,2497.129,2013,1,1,1,-8.3
1,2013-01-01,2,A,2363.265,2013,1,1,1,-8.5
2,2013-01-01,3,A,2258.505,2013,1,1,1,-8.4
3,2013-01-01,4,A,2243.969,2013,1,1,1,-8.1
4,2013-01-01,5,A,2344.105,2013,1,1,1,-8.2
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,H,681.033,2018,12,31,0,-3.1
368084,2018-12-31,21,H,669.961,2018,12,31,0,-3.7
368085,2018-12-31,22,H,657.941,2018,12,31,0,-4.6
368086,2018-12-31,23,H,610.953,2018,12,31,0,-5.4


In [ ]:
d_map = {}
for i,d in enumerate(total['구분'].unique()):
  d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [ ]:
total

,연월일,시간,구분,공급량,year,month,day,weekday,temp
0,2013-01-01,1,0,2497.129,2013,1,1,1,-8.3
1,2013-01-01,2,0,2363.265,2013,1,1,1,-8.5
2,2013-01-01,3,0,2258.505,2013,1,1,1,-8.4
3,2013-01-01,4,0,2243.969,2013,1,1,1,-8.1
4,2013-01-01,5,0,2344.105,2013,1,1,1,-8.2
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,-3.1
368084,2018-12-31,21,6,669.961,2018,12,31,0,-3.7
368085,2018-12-31,22,6,657.941,2018,12,31,0,-4.6
368086,2018-12-31,23,6,610.953,2018,12,31,0,-5.4


## Trian / Validation 구분
- 주의할 점 ! 전년도 자료(ex.전년도 공급량 등 shift해 오는 변수) 사용 시 2013은 제외

In [ ]:
train_years = [2013,2014,2015,2016,2017] 
val_years = [2018]

In [ ]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [ ]:
val

,연월일,시간,구분,공급량,year,month,day,weekday,temp
306768,2018-01-01,1,0,1765.008,2018,1,1,0,-3.2
306769,2018-01-01,2,0,1679.186,2018,1,1,0,-3.3
306770,2018-01-01,3,0,1610.885,2018,1,1,0,-3.7
306771,2018-01-01,4,0,1604.123,2018,1,1,0,-4.0
306772,2018-01-01,5,0,1711.506,2018,1,1,0,-4.2
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,-3.1
368084,2018-12-31,21,6,669.961,2018,12,31,0,-3.7
368085,2018-12-31,22,6,657.941,2018,12,31,0,-4.6
368086,2018-12-31,23,6,610.953,2018,12,31,0,-5.4


In [ ]:
features = ['구분','year', 'month', 'weekday','day', '시간', 'temp'] #생성한 모든 feature 지정
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

# 모델링 - LGBM / XGBOOST

## (1) LGBM Baseline

### LightGBM Feature Selection

In [ ]:
lgbm_features =  ['구분', 'month',  'weekday', '시간', 'temp']   #lgbm_feature 선택시 year, day 제거하는게 성능 제일 좋았음 & 과적합 낮춤
              
train_lgbm = train_x[lgbm_features]
val_lgbm = val_x[lgbm_features]

### LightGBM Hyperparameter


In [ ]:
params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

### 학습

In [ ]:
ds_train = lgb.Dataset(train_lgbm, train_y)
ds_val = lgb.Dataset(val_lgbm, val_y)

lgbm_model = lgb.train(params, ds_train, 20000, ds_val, verbose_eval=5000, early_stopping_rounds=2000)

Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[541]	valid_0's l1: 95.3623


In [ ]:
y_train_lgbm_pred = lgbm_model.predict(train_lgbm)
y_val_lgbm_pred = lgbm_model.predict(val_lgbm)

In [ ]:
#CV SCORE
print('Train nmae:', np.mean((np.abs(train_y - y_train_lgbm_pred))/train_y))
print('Validation nmae:', np.mean((np.abs(val_y - y_val_lgbm_pred))/val_y))

Train nmae: 0.5057842551385187
Validation nmae: 0.5373837089551435


In [ ]:
lgbm_prediction_df = pd.DataFrame(val['연월일'], columns=['연월일'])
lgbm_prediction_df['공급량'] = val['공급량']
lgbm_prediction_df['예측'] = y_val_lgbm_pred
lgbm_prediction_df

,연월일,공급량,예측
306768,2018-01-01,1765.008,1885.439383
306769,2018-01-01,1679.186,1639.775692
306770,2018-01-01,1610.885,1606.055155
306771,2018-01-01,1604.123,1658.085368
306772,2018-01-01,1711.506,1858.986013
...,...,...,...
368083,2018-12-31,681.033,648.057433
368084,2018-12-31,669.961,667.593063
368085,2018-12-31,657.941,652.055222
368086,2018-12-31,610.953,606.444955


## (2) XGBOOST Baseline

### XGBOOST Feature Selection

In [ ]:
xgb_features =  ['구분', 'month','weekday', '시간', 'temp']
              
xgb_train = train_x[xgb_features]
xgb_val = val_x[xgb_features]

### XGBOOST Hyperparameter

In [ ]:
params = {
    'learning_rate': 0.05,
    'objective': 'reg:squarederror',
    'eval_metric':'mae', 
    'seed':42
}

In [ ]:
import xgboost as xgb

d_train = xgb.DMatrix(data=xgb_train, label = train_y)
d_val = xgb.DMatrix(data=xgb_val, label = val_y)
wlist = [(d_train, 'train'), (d_val,'eval')]

xgb_model = xgb.train(params, d_train, 500, evals=wlist, verbose_eval=20, early_stopping_rounds=100)

[0]	train-mae:887.89923	eval-mae:967.24982
[20]	train-mae:326.98700	eval-mae:400.02499
[40]	train-mae:155.84535	eval-mae:214.73106
[60]	train-mae:109.06935	eval-mae:152.27632
[80]	train-mae:93.52300	eval-mae:128.07765
[100]	train-mae:85.03659	eval-mae:115.11136
[120]	train-mae:80.26866	eval-mae:108.33803
[140]	train-mae:77.24566	eval-mae:104.75452
[160]	train-mae:75.41416	eval-mae:102.90157
[180]	train-mae:74.06461	eval-mae:101.66499
[200]	train-mae:73.11219	eval-mae:100.84144
[220]	train-mae:72.18221	eval-mae:100.10778
[240]	train-mae:71.23539	eval-mae:99.44418
[260]	train-mae:70.44855	eval-mae:98.91412
[280]	train-mae:69.58240	eval-mae:98.40372
[300]	train-mae:69.10795	eval-mae:98.14090
[320]	train-mae:68.51848	eval-mae:97.87711
[340]	train-mae:68.11301	eval-mae:97.75459
[360]	train-mae:67.66412	eval-mae:97.56922
[380]	train-mae:67.37687	eval-mae:97.48392
[400]	train-mae:67.01594	eval-mae:97.24865
[420]	train-mae:66.71385	eval-mae:97.09623
[440]	train-mae:66.36204	eval-mae:96.89832
[

In [ ]:
xgb_train_pred = (xgb_model.predict(d_train))
xgb_val_pred = (xgb_model.predict(d_val))


In [ ]:
#CV SCORE
print('Train nmae:', np.mean((np.abs(train_y - xgb_train_pred))/train_y))
print('Validation nmae:', np.mean((np.abs(val_y - xgb_val_pred))/val_y))

Train nmae: 0.5177819501100315
Validation nmae: 0.5322925109348885


In [ ]:
xgb_prediction_df = pd.DataFrame(val['연월일'], columns=['연월일'])
xgb_prediction_df['공급량'] = val['공급량']
xgb_prediction_df['예측'] = xgb_val_pred
xgb_prediction_df

,연월일,공급량,예측
306768,2018-01-01,1765.008,1903.419922
306769,2018-01-01,1679.186,1616.419312
306770,2018-01-01,1610.885,1549.545532
306771,2018-01-01,1604.123,1619.035156
306772,2018-01-01,1711.506,1906.646729
...,...,...,...
368083,2018-12-31,681.033,665.103943
368084,2018-12-31,669.961,694.223938
368085,2018-12-31,657.941,654.115356
368086,2018-12-31,610.953,573.458069


## (3) catboost Baseline

### LightGBM Feature Selection

In [ ]:
cat = ['구분', 'month','weekday', '시간', 'temp']

cat_train = train_x[cat]
cat_val = val_x[cat]

### 학습

In [ ]:
cat_features = [0, 1, 7, 8]

catboost_model = CatBoostRegressor(
    iterations=20000,
    learning_rate=0.05,
    max_ctr_complexity=4,
    random_seed=0,
    od_type='Iter',
    early_stopping_rounds = 2000,
    verbose=5000,
    depth=4
)

catboost_model.fit(
    cat_train, train_y,
    cat_features=[],
    eval_set=(cat_val,val_y)
)

0:	learn: 875.6187727	test: 979.5465116	best: 979.5465116 (0)	total: 35.4ms	remaining: 11m 47s
5000:	learn: 113.9464478	test: 157.2477106	best: 157.2465862 (4997)	total: 2m 12s	remaining: 6m 36s
10000:	learn: 110.4486162	test: 156.9305793	best: 156.9285630 (9959)	total: 4m 24s	remaining: 4m 24s
Stopped by overfitting detector  (2000 iterations wait)

bestTest = 156.9243675
bestIteration = 10025

Shrink model to first 10026 iterations.


In [ ]:
cat_train_pred = catboost_model.predict(cat_train)
cat_val_pred = catboost_model.predict(cat_val)

In [ ]:
#CV SCORE
print('Train nmae:', np.mean((np.abs(train_y - cat_train_pred))/train_y))
print('Validation nmae:', np.mean((np.abs(val_y - cat_val_pred))/val_y))

Train nmae: 0.5043929593991439
Validation nmae: 0.5442653736660381


## automl

In [ ]:
!python3 -m pip install --upgrade pip
!pip3 install scikit-learn==0.24.1
!pip3 install pandas

In [ ]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
!pip3 install auto-sklearn

In [ ]:
!pip3 install auto-sklearn --upgrade

In [ ]:
pip install --upgrade scipy

In [ ]:
import scipy
print(scipy.__version__)

1.7.1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.metrics import mean_squared_error as auto_mean_squared_error
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score

In [ ]:
model = AutoSklearnRegressor(time_left_for_this_task=3600,
    per_run_time_limit=360,
    memory_limit=27000, metric=auto_mean_squared_error,)

In [ ]:
model.fit(train_x, train_y)

Process ForkProcess-6:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/autosklearn/util/logging_.py", line 320, in start_log_server
    receiver.serve_until_stopped()
  File "/usr/local/lib/python3.7/dist-packages/autosklearn/util/logging_.py", line 352, in serve_until_stopped
    self.timeout)
KeyboardInterrupt


In [ ]:
print(model.leaderboard())

### 플러스
- 기온 변수 말고 냉방도일 변수 추가해보기
- 구분이 `지역`or `공급사`라고 가정하고 구분 별로 그룹화시켜 따로 예측
- train에 변수가 추가되면 test에도 변수 칼럼 개수를 똑같이 맞춰야하는거 아닌가 //? 같이 추가해야만 하지 않는가 ?
- 90일 , 즉 1월1일부터 3월31일까지 예측이니만큼, 계절 추가 /절기 추가 /시간은 sin.cos사이클 돌리기 / 

#  결과 제출

In [ ]:
test = pd.read_csv(PATH + 'test.csv')
submission = pd.read_csv(PATH + 'sample_submission.csv')

## test data
- 주의 ! train에서 선택한 피쳐와 동일해야 함

In [ ]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [ ]:
test

,일자|시간|구분,일자,시간,구분
0,2019-01-01 01 A,2019-01-01,1,A
1,2019-01-01 02 A,2019-01-01,2,A
2,2019-01-01 03 A,2019-01-01,3,A
3,2019-01-01 04 A,2019-01-01,4,A
4,2019-01-01 05 A,2019-01-01,5,A
...,...,...,...,...
15115,2019-03-31 20 H,2019-03-31,20,H
15116,2019-03-31 21 H,2019-03-31,21,H
15117,2019-03-31 22 H,2019-03-31,22,H
15118,2019-03-31 23 H,2019-03-31,23,H


In [ ]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday
test['구분'] = test['구분'].map(d_map)

In [ ]:
test_x = test[['구분', 'month', 'weekday', '시간']] 

In [ ]:
test_x

,구분,month,weekday,시간
0,0,1,1,1
1,0,1,1,2
2,0,1,1,3
3,0,1,1,4
4,0,1,1,5
...,...,...,...,...
15115,6,3,6,20
15116,6,3,6,21
15117,6,3,6,22
15118,6,3,6,23


##(1) LGBM 모델링 결과 제출

In [ ]:
preds = (lgbm_model.predict(test_x))

In [ ]:
submission['공급량'] = preds

In [ ]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,1840.996416
1,2019-01-01 02 A,1621.531075
2,2019-01-01 03 A,1512.312396
3,2019-01-01 04 A,1551.481786
4,2019-01-01 05 A,1737.120489
...,...,...
15115,2019-03-31 20 H,526.270534
15116,2019-03-31 21 H,499.073871
15117,2019-03-31 22 H,440.105035
15118,2019-03-31 23 H,409.132005


In [ ]:
submission.to_csv(PATH + 'lgbm_기온추가.csv', index=False)

In [ ]:
total['temp'] = weather_total['기온(°C)'].values

,연월일,시간,구분,공급량,year,month,day,weekday,temp
0,2013-01-01,1,0,2497.129,2013,1,1,1,-8.3
1,2013-01-01,2,0,2363.265,2013,1,1,1,-8.5
2,2013-01-01,3,0,2258.505,2013,1,1,1,-8.4
3,2013-01-01,4,0,2243.969,2013,1,1,1,-8.1
4,2013-01-01,5,0,2344.105,2013,1,1,1,-8.2
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,-3.1
368084,2018-12-31,21,6,669.961,2018,12,31,0,-3.7
368085,2018-12-31,22,6,657.941,2018,12,31,0,-4.6
368086,2018-12-31,23,6,610.953,2018,12,31,0,-5.4


##(2) XGBOOST 모델링 결과 제출

In [ ]:
test_x = test[['구분', 'month', 'weekday', '시간']] 
test_x['temp'] = 0   #보간 필요 ex.전년도 온도 반영
test_x = xgb.DMatrix(test_x)

preds = (xgb_model.predict(test_x))

submission['공급량'] = preds

In [ ]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,1879.130493
1,2019-01-01 02 A,1620.813110
2,2019-01-01 03 A,1492.041748
3,2019-01-01 04 A,1545.896606
4,2019-01-01 05 A,1757.148804
...,...,...
15115,2019-03-31 20 H,537.707458
15116,2019-03-31 21 H,518.416260
15117,2019-03-31 22 H,467.967285
15118,2019-03-31 23 H,415.598114


In [ ]:
submission.to_csv('/content/drive/MyDrive/21-2 kubig 분류예측스터디/데이콘 가스공급량/제출용/' + 'xgb_기온추가_1031.csv', index=False)